# F8 Disaster Response ETL Pipeline

## Overview

This notebook aims to provide initial data exploration, clean and prepare the data for Text Classification model training later on. Few items we need:

* Messages, all in english and should have null and duplicates
* Categories on its own columns
* Both are combined into one table
* Saved in SQLite Database


## Preparation

Install and import required libraries

In [1]:
!{sys.executable} -m pip install -r ../requirements.txt -q

In [2]:
import pandas as pd
import numpy as np

from sqlalchemy import create_engine

## Import libraries and load datasets
- Load `messages.csv` into a dataframe and inspect the first few lines.
- Load `categories.csv` into a dataframe and inspect the first few lines.

In [3]:
# load messages dataset
messages = pd.read_csv("../data/raw/disaster_messages.csv")
messages.head()

FileNotFoundError: [Errno 2] No such file or directory: '../data/raw/messages.csv'

In [ ]:
messages.info()

Missing values only on `original` column. Makes sense as this column gets filled only if the language is not english.

In [ ]:
categories = pd.read_csv("../data/raw/disaster_categories.csv")
categories.head()

Categories column are in this format `related-1;request-0...`. This should be parsed into its own columns. 

In [ ]:
categories.info()

No missing values

## Merge datasets
- Merge the messages and categories datasets using the common id
- Assign this combined dataset to `df`, which will be cleaned in the following steps

In [ ]:
df = pd.merge(messages, categories, on='id')
df.head()

## Split `categories` into separate category columns
- Split the values in the `categories` column on the `;` character so that each value becomes a separate column. You'll find [this method](https://pandas.pydata.org/pandas-docs/version/0.23/generated/pandas.Series.str.split.html) very helpful! Make sure to set `expand=True`.
- Use the first row of categories dataframe to create column names for the categories data.
- Rename columns of `categories` with new column names.

In [ ]:
categories = df.categories.str.split(";", expand=True)
categories.head()

In [ ]:
row = categories.iloc[0]

row = row.apply(lambda col: col[:-2])

category_colnames = row.to_list()
print(category_colnames)

In [ ]:
categories.columns = category_colnames
categories.head()

## Convert category values to just numbers 0 or 1
- Iterate through the category columns in df to keep only the last character of each string (the 1 or 0). For example, `related-0` becomes `0`, `related-1` becomes `1`. Convert the string to a numeric value.

In [ ]:
for column in categories:
    # we need to ensure that no other values are zeroes and ones
    categories[column] = categories[column].apply(lambda col: col[-1]).apply(lambda col: "0" if col == "0" else "1") 
    
    categories[column] = pd.to_numeric(categories[column])
    
categories.head()

In [ ]:
# check columns that has a single value
one_unique_cols = categories.loc[:, categories.nunique() == 1].columns.to_list()
one_unique_cols

In [ ]:
categories.child_alone.value_counts()

Column `child_alone` has only 0 values. This will not help in the classification.

In [ ]:
categories = categories.drop(columns=one_unique_cols)

## Replace `categories` column in `df` with new category columns
- Drop the categories column from the df dataframe since it is no longer needed.
- Concatenate df and categories data frames.

In [ ]:
# drop the original categories column from `df`
df = df.drop(columns=["categories"])
df = pd.concat([df, categories], axis=1)

df.head()

In [ ]:
df.info()

## Remove duplicates
- Check how many duplicates are in this dataset.
- Drop the duplicates.
- Confirm duplicates were removed.

In [ ]:
# id is the most obvious way to determine duplicates
df.duplicated(subset=["id"]).sum()

In [ ]:
# drop duplicates in ID
df = df.drop_duplicates(subset=["id"])
df.duplicated(subset=["id"]).sum()

In [ ]:
# duplicate message can mess up prediction
df[df.duplicated(subset=["message"])]

This looks like an error exported from `Excel`. Content does not make sense so it's safe to delete them.

In [ ]:
# remove with message #NAME? as it doesn't make any sense

df = df[df.message != "#NAME?"]
df[df.duplicated(subset=["message"])]

## Save the clean dataset into an sqlite database.

In [ ]:
engine = create_engine('sqlite:///../data/processed/DisasterResponse.db')
df.to_sql('Message', engine, if_exists="replace", index=False)